In [2]:
from imdb import Cinemagoer
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import traceback
import networkx as nx
import json


this file started out as something to initally explore the data 

it later turned into into a file which gets movies from imdb

In [3]:
main_data = pd.read_csv("/home/sindri/mastersnstuff/social/warplan_surival/raw_data/scifi.csv")
ia = Cinemagoer()


In [4]:
main_data

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204965.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295177.0,NaN
2,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327912.0,NaN
3,tt8760708,M3GAN,2022,PG-13,102 min,"Horror, Sci-Fi, Thriller",6.4,A robotics engineer at a toy company builds a ...,Gerard Johnstone,/name/nm2067421/,"Allison Williams, \nViolet McGraw, \nRonny Chi...","/name/nm4129745/,/name/nm8627157/,/name/nm5307...",62493.0,NaN
4,tt10954600,Ant-Man and the Wasp: Quantumania,2023,PG-13,125 min,"Action, Adventure, Comedy",6.6,"Scott Lang and Hope Van Dyne, along with Hank ...",Peyton Reed,/name/nm0715636/,"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo...","/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5707.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16552,tt26733154,The Monsters in our Shadows,NaN,NaN,NaN,"Drama, Horror, Sci-Fi",NaN,A man must travel into the dark to bring light...,NaN,NaN,NaN,NaN,NaN,NaN
16553,tt26733855,#SHE,2018,NaN,95 min,"Horror, Sci-Fi",NaN,#SHE is a wild journey through the warped mind...,Arantxa Echevarría,/name/nm0248401/,"David James, \nNajarra Townsend","/name/nm14562731/,/name/nm1680142/",NaN,NaN
16554,tt26741527,Upgrade My Country: Rise of the Beast,2023,NaN,NaN,"Drama, Sci-Fi, Thriller",NaN,Add a Plot,Ben Baker-Lee,/name/nm3836602/,"Ben Baker-Lee, \nRichard K Baylor, \nIan Charl...","/name/nm3836602/,/name/nm14566875/,/name/nm145...",NaN,NaN
16555,tt26743984,Suicide 2,1999,NaN,54 min,"Action, Drama, Horror",NaN,Matt awakens after trying to end his own life ...,"Brandon Womeldorff, \nMatt Brassfield",/name/nm13690776/,"Matt Hoffman, \nMatt Brassfield, \nBrandon Wom...","/name/nm1565857/,/name/nm7326486/,/name/nm1565...",NaN,NaN


In [12]:

department_names = pd.read_csv("movie_departments.csv")["person"]
output_folder = "/home/sindri/mastersnstuff/social/warplan_surival/social_media_analytics/output_movies"


In [13]:
def get_departments(movie_obj):

    departments = {}
    for dept in department_names:
        departments[dept] = []
        try:
            for person in movie_obj[dept]:
                    departments[dept].append(person["name"])
        except:
            pass

    return departments


In [16]:

def get_movie_people(movie_dataframe : pd.DataFrame, output_folder : str):
    # based off of a presliced dataframe chunk
    # get the cast based of of that dataframes imdb ids infromation put it into json
    # and write that data to a file
    pos = 0
    movie_ids = movie_dataframe["movie_id"]
    while  pos < len(movie_ids):
        try: 
            movie_id = movie_ids.iloc[pos]
            movie = ia.get_movie(movie_id[2:])
            print(movie)
            departments = get_departments(movie)
            departments["name"] = movie.get('title') 
            output = json.dumps(departments, indent=4)
            name = movie_dataframe.iloc[pos]['movie_name'].replace(" ","_").replace("/","_")
            with open(f"{output_folder}/{name}.json", "w") as outfile:
                outfile.write(output)
            pos += 1
        except:
            traceback.print_exc()
            pass

In [ ]:
from multiprocessing import Process
import numpy as np

chunks = np.array_split(main_data, 10)

func_inputs = [ (df_chunk, output_folder)for df_chunk in chunks]
print(func_inputs)

proccess = []
for func_input in func_inputs:
    p = Process(target=get_movie_people, args=func_input)
    p.start()
    proccess.append(p)
        
for p in proccess:
    p.join()
    




